In [1]:
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pickle

In [2]:
rent = "final.csv"
rent_df = pd.read_csv(rent)
rent_df = rent_df.drop(["Unnamed: 0"], axis = 1)
rent_df.head()

,Property_Code,Tenant_Code,Event,DateOccurred,Unit_Code,DateMoveIn,DateMoveOut,Rent,Vacancy,SQFT,BedRooms,Bathroom,chrent,LengthStay
0,ce,t0013745,Rent Change,4/30/2013,C05266,11/9/2006,NaN,870.0,0,1400,3,2.5,870.0,NaN
1,ce,t0013745,Rent Change,5/31/2014,C05266,11/9/2006,1/31/2015,896.0,1,1400,3,2.5,26.0,245 days 00:00:00.000000000
2,ce,t0013747,Rent Change,4/30/2013,C05270,7/16/2012,8/11/2013,770.0,1,1088,2,1.5,770.0,103 days 00:00:00.000000000
3,ce,t0013748,Rent Change,4/30/2013,C05272,9/28/2012,NaN,770.0,0,1088,2,1.5,770.0,NaN
4,ce,t0013748,Rent Change,9/30/2013,C05272,9/28/2012,NaN,800.0,0,1088,2,1.5,30.0,NaN


In [83]:
sqft_bins = [0, 600, 1000, 2000]
bin_names = ["small", "medium", "large"]
rent_df["Apt Size"] = pd.cut(rent_df["SQFT"], sqft_bins, labels = bin_names)
indexNames = rent_df[rent_df["Rent"] == rent_df ["chrent"]].index
rent_df.drop(indexNames, inplace = True)
outliers = rent_df[rent_df["chrent"] >= 500].index
rent_df.drop(outliers, inplace = True)
outliers2 = rent_df[rent_df["chrent"] <= -500].index
rent_df.drop(outliers2, inplace = True)
rent_df

,Property_Code,Tenant_Code,Event,DateOccurred,Unit_Code,DateMoveIn,DateMoveOut,Rent,Vacancy,SQFT,BedRooms,Bathroom,chrent,LengthStay,Apt Size
1,ce,t0013745,Rent Change,5/31/2014,C05266,11/9/2006,1/31/2015,896.0,1,1400,3,2.5,26.0,245 days 00:00:00.000000000,large
4,ce,t0013748,Rent Change,9/30/2013,C05272,9/28/2012,NaN,800.0,0,1088,2,1.5,30.0,NaN,large
5,ce,t0013748,Rent Change,11/1/2014,C05272,9/28/2012,10/31/2015,824.0,1,1088,2,1.5,24.0,364 days 00:00:00.000000000,large
9,ce,t0013749,Rent Change,12/31/2013,C05274,11/26/2008,NaN,915.0,0,1400,3,2.5,35.0,NaN,large
10,ce,t0013749,Rent Change,3/1/2015,C05274,11/26/2008,NaN,924.0,0,1400,3,2.5,9.0,NaN,large
11,ce,t0013749,Rent Change,2/29/2016,C05274,11/26/2008,NaN,970.0,0,1400,3,2.5,46.0,NaN,large
12,ce,t0013749,Rent Change,2/28/2018,C05274,11/26/2008,NaN,1044.0,0,1400,3,2.5,74.0,NaN,large
14,ce,t0013750,Rent Change,6/30/2013,C05276,7/21/2001,NaN,890.0,0,1400,3,2.5,10.0,NaN,large
15,ce,t0013750,Rent Change,6/30/2014,C05276,7/21/2001,NaN,917.0,0,1400,3,2.5,27.0,NaN,large
16,ce,t0013750,Rent Change,6/30/2015,C05276,7/21/2001,NaN,945.0,0,1400,3,2.5,28.0,NaN,large


In [84]:
X = rent_df[["Apt Size", "BedRooms", "Bathroom", "chrent", "Property_Code"]]
y = rent_df["Vacancy"]
print(X.shape, y.shape)

(5950, 5) (5950,)


In [85]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data, columns = ["Apt Size", "Property_Code"])
data_binary_encoded

,BedRooms,Bathroom,chrent,Apt Size_small,Apt Size_medium,Apt Size_large,Property_Code_ce,Property_Code_fh,Property_Code_mpg,Property_Code_sh
1,3,2.5,26.0,0,0,1,1,0,0,0
4,2,1.5,30.0,0,0,1,1,0,0,0
5,2,1.5,24.0,0,0,1,1,0,0,0
9,3,2.5,35.0,0,0,1,1,0,0,0
10,3,2.5,9.0,0,0,1,1,0,0,0
11,3,2.5,46.0,0,0,1,1,0,0,0
12,3,2.5,74.0,0,0,1,1,0,0,0
14,3,2.5,10.0,0,0,1,1,0,0,0
15,3,2.5,27.0,0,0,1,1,0,0,0
16,3,2.5,28.0,0,0,1,1,0,0,0


In [86]:
data_binary_encoded = data_binary_encoded.fillna(0)

In [87]:
data_binary_encoded.dtypes

BedRooms               int64
Bathroom             float64
chrent               float64
Apt Size_small         uint8
Apt Size_medium        uint8
Apt Size_large         uint8
Property_Code_ce       uint8
Property_Code_fh       uint8
Property_Code_mpg      uint8
Property_Code_sh       uint8
dtype: object

In [88]:
data_binary_encoded ["BedRooms"]= data_binary_encoded["BedRooms"].astype(float)
data_binary_encoded ["Apt Size_small"]= data_binary_encoded["Apt Size_small"].astype(float)
data_binary_encoded ["Apt Size_medium"]= data_binary_encoded["Apt Size_medium"].astype(float)
data_binary_encoded ["Apt Size_large"]= data_binary_encoded["Apt Size_large"].astype(float)
data_binary_encoded ["Property_Code_ce"]= data_binary_encoded["Property_Code_ce"].astype(float)
data_binary_encoded ["Property_Code_fh"]= data_binary_encoded["Property_Code_fh"].astype(float)
data_binary_encoded ["Property_Code_mpg"]= data_binary_encoded["Property_Code_mpg"].astype(float)
data_binary_encoded ["Property_Code_sh"]= data_binary_encoded["Property_Code_sh"].astype(float)

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(data_binary_encoded, y, stratify=y, random_state=42)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [90]:
X_test.shape

(1488, 10)

In [91]:
from keras.models import Sequential
from keras.layers import Dense
deep_model = Sequential()

deep_model.add(Dense(units=100, activation='relu', input_dim=10))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [92]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 0s - loss: 0.6183 - acc: 0.7344
Epoch 2/100
 - 0s - loss: 0.5824 - acc: 0.7398
Epoch 3/100
 - 0s - loss: 0.5717 - acc: 0.7400
Epoch 4/100
 - 0s - loss: 0.5671 - acc: 0.7423
Epoch 5/100
 - 0s - loss: 0.5666 - acc: 0.7416
Epoch 6/100
 - 0s - loss: 0.5610 - acc: 0.7456
Epoch 7/100
 - 0s - loss: 0.5617 - acc: 0.7450
Epoch 8/100
 - 0s - loss: 0.5604 - acc: 0.7445
Epoch 9/100
 - 0s - loss: 0.5569 - acc: 0.7470
Epoch 10/100
 - 0s - loss: 0.5596 - acc: 0.7450
Epoch 11/100
 - 0s - loss: 0.5579 - acc: 0.7454
Epoch 12/100
 - 0s - loss: 0.5590 - acc: 0.7414
Epoch 13/100
 - 0s - loss: 0.5542 - acc: 0.7456
Epoch 14/100
 - 0s - loss: 0.5545 - acc: 0.7445
Epoch 15/100
 - 0s - loss: 0.5518 - acc: 0.7474
Epoch 16/100
 - 0s - loss: 0.5507 - acc: 0.7474
Epoch 17/100
 - 0s - loss: 0.5537 - acc: 0.7454
Epoch 18/100
 - 0s - loss: 0.5521 - acc: 0.7483
Epoch 19/100
 - 0s - loss: 0.5513 - acc: 0.7485
Epoch 20/100
 - 0s - loss: 0.5472 - acc: 0.7485
Epoch 21/100
 - 0s - loss: 0.5527 - acc: 0.7465
E

In [93]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Deep Neural Network - Loss: 0.5475274285962505, Accuracy: 0.7473118279569892


In [94]:
deep_predictions = deep_model.predict_classes(X_test)
deep_predictor = pd.DataFrame({"Predictions": deep_predictions, "Actual": y_test})
deep_predictor.loc[deep_predictor["Predictions"] == 1, "Predictions"] = "Vacant"
deep_predictor.loc[deep_predictor["Predictions"] == 0, "Predictions"] = "Occupied"
deep_predictor.loc[deep_predictor["Actual"] == 1, "Actual"] = "Vacant"
deep_predictor.loc[deep_predictor["Actual"] == 0, "Actual"] = "Occupied"
deep_predictor

,Predictions,Actual
4743,Occupied,Vacant
6500,Occupied,Occupied
2414,Occupied,Vacant
5931,Occupied,Occupied
844,Occupied,Occupied
2777,Occupied,Occupied
6036,Occupied,Occupied
3137,Occupied,Occupied
1334,Occupied,Occupied
1692,Occupied,Occupied


In [69]:
deep_model.save("logistics_deep_network.h5")

In [101]:
filename = "logisitic_depp_network.pkl"
list_pickle = open(filename, "wb")
pickle.dump(deep_model, list_pickle)
list_pickle.close()